# Load Configurations & Libraries
Creating Vector Database

In [ ]:
# If we run this machine on GPU machine it will be faster.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
apipath = r'H:\\My Drive\\config\\hbqa.txt'
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
PINECONE_API_KEY = config['global']['PINECONE_API_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

### Load Embedding Model. It will create 384 dim vector for the sentences.

In [ ]:
!pip install -Uq sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Creating Embedding for Chunks

In [6]:
df_hbqa_chunk = pd.read_csv(datapath+"05-QAGen_ChatGPT_FE-Created-14Sep23-(ManualStep).csv")
print(df_hbqa_chunk.shape)
df_hbqa_chunk.head(4)

(256, 7)


,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,2000,2030,Book01_046,10193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,2000,3462,Book01_104,10425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2000,1974,Book01_007,10083,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...
3,2000,1492,Book01_005,10078,"Write 1 question, corresponding answer, and th...",And the Rakshasa saw the apartment in which th...,Q1: What did the Rakshasa ask the god of fire ...


In [9]:
df_hbqa_chunk.head(4)

,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,2000,2030,Book01_046,10193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,2000,3462,Book01_104,10425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2000,1974,Book01_007,10083,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...
3,2000,1492,Book01_005,10078,"Write 1 question, corresponding answer, and th...",And the Rakshasa saw the apartment in which th...,Q1: What did the Rakshasa ask the god of fire ...


In [14]:
DEVICE='cuda'

In [15]:
%time
Chunk_Sentences = df_hbqa_chunk.Chunk.tolist()
Chunk_Embeddings = model.encode(Chunk_Sentences)

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 6.68 µs


In [16]:
import torch

In [17]:
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

In [18]:
# Flatten the tensors into 1D arrays
ChunkVec_list = ChunkVectors.tolist()

In [19]:
df_hbqa_chunk['ChunkVector'] = ChunkVectors.tolist()

In [20]:
df_hbqa_chunk.reset_index(inplace=True)

In [ ]:
df_hbqa_chunk.to_csv(datapath + '07.1-HBQA_Chunk_Vector.csv')

In [36]:
df_hbqa_chunk.head(3)

,index,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,0,2000.0,2030.0,Book01_046,10193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,1,2000.0,3462.0,Book01_104,10425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2,2000.0,1974.0,Book01_007,10083,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...


# Creat Embedding for Question & Answer


In [13]:
# df_hbqa_auto_qa = pd.read_csv(datapath+"06.1-HBQA_Auto_with_Chunk.csv")
df_hbqa_manul_qa = pd.read_csv(datapath+"06-HBQA_Manual_with_Chunk.csv")


In [14]:
# df_hbqa_auto_qa.shape,
df_hbqa_manul_qa.shape

(1104, 9)

In [15]:
df_hbqa_manul_qa.head(2)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns
0,10000,10193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...",14,25
1,10001,10193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...",10,26


In [18]:
DEVICE='cuda'

In [16]:
%time
Ques_Sentences = df_hbqa_manul_qa.Question.tolist()
Ques_Embeddings = model.encode(Ques_Sentences)

Ans_Sentences = df_hbqa_manul_qa.Ref_Answer.tolist()
Ans_Embeddings = model.encode(Ans_Sentences)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [20]:
import torch

In [21]:
QuesVectors= torch.tensor(Ques_Embeddings, dtype=torch.float).to(DEVICE)
AnsVectors= torch.tensor(Ans_Embeddings, dtype=torch.float).to(DEVICE)

In [22]:
df_hbqa_Vector = df_hbqa_manul_qa.copy()

In [23]:
df_hbqa_Vector.drop(columns = ['WordsInQues','WordsInAns'], axis=1, inplace=True)

In [24]:
# Flatten the tensors into 1D arrays and save
df_hbqa_Vector['QuesVector'] = QuesVectors.tolist()
df_hbqa_Vector['AnsVector'] = AnsVectors.tolist()

In [25]:
df_hbqa_Vector.head(2)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,QuesVector,AnsVector
0,10000,10193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...","[0.024200892075896263, 0.11250999569892883, 0....","[0.0017887662397697568, 0.10703910887241364, 0..."
1,10001,10193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...","[-0.08296065777540207, 0.14385125041007996, 0....","[-0.061754725873470306, 0.13033652305603027, 0..."


In [26]:
df_hbqa_Vector.to_csv(datapath + '07.2-HBQA_QA_Vector.csv')

# Save Context/Chunk Vector into Pinecone Vector database

In [ ]:
!pip install -Uq pinecone-client

In [ ]:
# import pinecone

# pinecone.init(
# 	api_key='ddf2ef12-9108-4e80-b438-8f86a0b617c0',
# 	environment='asia-southeast1-gcp-free'
# )
# index = pinecone.Index('HBQA')

In [7]:
import os
import pinecone

# # get api key from app.pinecone.io
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# # find your environment next to the api key in pinecone console
# PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# index_name = 'semantic-search-fast'
index_name = "hbqa" #it must be in lower letters

In [ ]:
# !pip uninstall pinecone_datasets

In [ ]:
# from pinecone_datasets import load_dataset

# dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# # we drop sparse_values as they are not needed for this example
# dataset.documents.drop(['metadata'], axis=1, inplace=True)
# dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# # we will use 80K rows of the dataset between rows 240K -> 320K
# dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
# dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
# dataset.head()

In [ ]:
import ast
import time

In [9]:
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(ast.literal_eval(df_hbqa_chunk['ChunkVector'][0])),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

In [8]:
# now connect to the index
index = pinecone.Index(index_name)

In [ ]:
chunk_id = df_hbqa_chunk['Chunk_Id'].to_list()

chunk_vector = [df_hbqa_chunk['ChunkVector'][i] for i in range(df_hbqa_chunk.shape[0])]

In [ ]:
import random
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# vector_dim = 128
# vector_count = 10000

# Example generator that generates many (id, vector) pairs
# example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))
data_generator = map(lambda i: ( 'id'+str(chunk_id[i]), chunk_vector[i]),  range(df_hbqa_chunk.shape[0]))

# Upsert data with 100 vectors per upsert request
i=1
batchsize = 100
for ids_vectors_chunk in chunks(data_generator, batch_size=batchsize):
    index.upsert(vectors=ids_vectors_chunk)  # Assuming `index` defined elsewhere
    print("Processed ",i*batchsize, " Vectors")
    i+=1


Processed  100  Vectors
Processed  200  Vectors
Processed  300  Vectors
Processed  400  Vectors
Processed  500  Vectors
Processed  600  Vectors


# Check whether a correct document can be retrived using a Question

In [ ]:
from random import randint
i = randint(0, df_hbqa_chunk.shape[0])
df_hbqa_chunk.loc[i,:]

index                                                        251
Context_Len                                               6000.0
Len                                                       5375.0
Section_Id                                            Book01_002
Chunk_Id                                                   10017
Prompt         Write 3 questions, corresponding answers, and ...
Chunk          Then shall I describe the Parva called Sauptik...
QA             Q1: What incidents are described in the Saupti...
Name: 251, dtype: object

In [ ]:
df_hbqa_chunk.loc[i,'QA']

'Q1: What incidents are described in the Sauptika Parva?\nA1: The Sauptika Parva narrates the entry of Kritavarman, Kripa, and Aswatthaman into the battlefield after the departure of the Pandavas. It includes the slaughter of the Panchalas and Draupadi\'s distress. Aswatthaman attempts to kill the sleeping Panchalas but is stopped by Krishna and Arjuna. Krishna and Dwaipayana (Vyasa) curse Aswatthaman, and Pandava takes away the jewel from his head.\nR1: "Then shall I describe the Parva called Sauptika of frightful incidents." - (Context)\n\nQ1: What events take place in the Stri Parva?\nA1: The Stri Parva depicts Dhritarashtra\'s anguish over the death of his children and his enmity towards Bhima. Vidura consoles Dhritarashtra and then describes their visit to the battlefield. The wives of the slain heroes lament, and Gandhari and Dhritarashtra become unconscious. Krishna pacifies Gandhari, and the bodies of the deceased princes are cremated. The Parva also includes the revelation of 

In [ ]:
sep=":"
query = df_hbqa_chunk.loc[i,'QA'].split('?')[0].split(sep)[1]
print(query)

# create the query vector
xq = model.encode(query).tolist()
topk = 5
# now query
xc = index.query(xq, top_k=topk, include_metadata=True)
xc

 What incidents are described in the Sauptika Parva


{'matches': [{'id': 'id10017', 'score': 0.455262512, 'values': []},
             {'id': 'id11', 'score': 0.437598526, 'values': []},
             {'id': 'id8', 'score': 0.431152731, 'values': []},
             {'id': 'id10692', 'score': 0.430405021, 'values': []},
             {'id': 'id10009', 'score': 0.403491229, 'values': []}],
 'namespace': ''}

In [ ]:
predictedId=[]
for predid in xc['matches']:
  predictedId.append(predid['id'])

print ("predicted chunkid ", predictedId, " Actual Chunk Id:", df_hbqa_chunk.loc[i,'Chunk_Id'])

predicted chunkid  ['id10017', 'id11', 'id8', 'id10692', 'id10009']  Actual Chunk Id: 10017
